In [1]:
import pandas as pd
import numpy as np

In [19]:
train_data=pd.read_csv("data/train.csv")
train_data.head(3)

,id,zipcode,score
0,2,8810,11
1,3,28129,4
2,4,79915,25


In [16]:
zip_info=pd.read_csv("data/zipcode2016/16zpallagi.csv")
zip_info.columns

Index(['STATEFIPS', 'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2',
       'MARS4', 'PREP', 'N2',
       ...
       'N10300', 'A10300', 'N85530', 'A85530', 'N85300', 'A85300', 'N11901',
       'A11901', 'N11902', 'A11902'],
      dtype='object', length=147)

In [5]:
zip_info["zipcode"].nunique()

29874

In [6]:
zip_info.shape

(179796, 147)

In [14]:
zip_info[zip_info["zipcode"]==28129]

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
114991,37,NC,28129,1,800,550,140,100,470,1080,...,410,402,0,0,0,0,80,101,670,1116
114992,37,NC,28129,2,670,300,230,130,410,1330,...,570,1518,0,0,0,0,110,153,560,1359
114993,37,NC,28129,3,370,100,230,40,260,860,...,360,1873,0,0,0,0,80,177,290,717
114994,37,NC,28129,4,280,30,250,0,210,760,...,280,2264,0,0,0,0,80,223,200,681
114995,37,NC,28129,5,290,20,280,0,200,850,...,290,4495,0,0,0,0,100,431,190,754
114996,37,NC,28129,6,30,0,20,0,20,70,...,30,4466,0,0,0,0,20,1002,0,0


In [12]:
train_data[train_data["zipcode"]==]

,id,zipcode,score


In [19]:
zip_info1=zip_info.groupby(by=["zipcode"]).count()

In [23]:
zip_info1[zip_info1["agi_stub"]==4]

,STATEFIPS,STATE,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
zipcode,,,,,,,,,,,,,,,,,,,,,
16802,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [24]:
train_data["zipcode"].nunique()

23788

In [25]:
train_data.shape

(23788, 3)

In [26]:
return_name=zip_info.loc[:, "N1":"A11902"].columns

In [29]:
distinct_name=zip_info.loc[:, "STATEFIPS":"zipcode"].columns

In [31]:
to_num={1:12500, 2:37500, 3:62500, 4:87500, 5:150000, 6:212500}
zip_info["avg_income"]=zip_info["agi_stub"].apply(lambda x: to_num[x])
zip_info["agg_income"]=zip_info["avg_income"]*zip_info["N1"]

In [32]:
zip_info_agg=zip_info.groupby(by="zipcode")["agg_income", "N1"].sum()
zip_info_agg.head(5)

,agg_income,N1
zipcode,,
0,8626236500000,147025790
1001,521875000,8980
1002,677625000,9490
1003,2625000,170
1005,143500000,2370


In [33]:
zip_info_agg.reset_index(inplace=True)
zip_info_agg["income_level"]=zip_info_agg["agg_income"]/zip_info_agg["N1"]
zip_info_agg.head(5)

,zipcode,agg_income,N1,income_level
0,0,8626236500000,147025790,58671.587481
1,1001,521875000,8980,58115.256125
2,1002,677625000,9490,71404.109589
3,1003,2625000,170,15441.176471
4,1005,143500000,2370,60548.523207


In [34]:
train_data=train_data.merge(zip_info_agg, how="left", on="zipcode")
train_data.head(5)

,id,zipcode,score,agg_income,N1,income_level
0,2,8810,11,341750000,3830,89229.765013
1,3,28129,4,132625000,2440,54354.508197
2,4,79915,25,421000000,15510,27143.778208
3,5,21085,10,571625000,8690,65779.631761
4,6,62882,1,38875000,950,40921.052632


In [35]:
train_data.drop("agg_income", axis=1, inplace=True)
train_data.drop("N1", axis=1, inplace=True)
train_data.head(5)

,id,zipcode,score,income_level
0,2,8810,11,89229.765013
1,3,28129,4,54354.508197
2,4,79915,25,27143.778208
3,5,21085,10,65779.631761
4,6,62882,1,40921.052632


In [36]:
zip_info_agg2=zip_info.groupby(by="zipcode").sum()
train_data=train_data.merge(zip_info_agg2.loc[:, "N1":"A11902"], how="left", on="zipcode")
train_data.columns

Index(['id', 'zipcode', 'score', 'income_level', 'N1', 'mars1', 'MARS2',
       'MARS4', 'PREP', 'N2',
       ...
       'N10300', 'A10300', 'N85530', 'A85530', 'N85300', 'A85300', 'N11901',
       'A11901', 'N11902', 'A11902'],
      dtype='object', length=147)

In [20]:
train_data_id=train_data.loc[:, ["id", "zipcode"]]
train_data.drop("id", axis=1, inplace=True)
train_data.drop("zipcode", axis=1, inplace=True)
train_data.head(3)

,score
0,11
1,4
2,25


In [38]:
train_data.to_csv("data/experiment.csv", index=False)

In [2]:
train_data=pd.read_csv("data/experiment.csv")
train_data.head(3)

,score,income_level,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,11,89229.765013,3830,1520,1980,280,2250,8290,2890,50,...,3240,64384,230,477,180,197,850,4356,2800,10143
1,4,54354.508197,2440,1000,1150,270,1570,4950,1510,100,...,1940,15018,0,0,0,0,470,2087,1910,4627
2,25,27143.778208,15510,7320,4010,4060,8180,31310,12630,670,...,9580,25892,0,0,0,0,1380,2427,13870,41617


In [9]:
temp=train_data.loc[:, "mars1":"ELDERLY"].columns

In [10]:
for column in temp:
    train_data["%s_N1" %column]=train_data[column]/train_data["N1"]

In [11]:
temp=train_data.loc[:, "NUMDEP":"ELDERLY"].columns
for column in temp:
    train_data["%s_N2" %column]=train_data[column]/train_data["N2"]

In [12]:
temp=train_data.loc[:, "A02650":"A11902"].columns
for column in temp:
    if column[0]=='A':
        train_data["%s_A00100" %column]=train_data[column]/train_data["A00100"]
    else:
        train_data["%s_N02650" %column]=train_data[column]/train_data["N02650"]

In [13]:
train_data.to_csv("data/experiment_1.csv", index=False)

In [54]:
train_data=pd.read_csv("data/experiment_1.csv")

In [55]:
train_data["zipcode"]=train_data_id["zipcode"]

In [56]:
zip_info.columns

Index(['STATEFIPS', 'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2',
       'MARS4', 'PREP', 'N2',
       ...
       'N10300', 'A10300', 'N85530', 'A85530', 'N85300', 'A85300', 'N11901',
       'A11901', 'N11902', 'A11902'],
      dtype='object', length=147)

In [57]:
for i in range(6):
    level=i+1
    
    zip_info_lvl=zip_info[zip_info["agi_stub"]==level]
    #print(zip_info_lvl.columns)
    #print(zip_info_lvl.head(1))
    zip_info_lvl.drop("STATEFIPS", axis=1, inplace=True)
    zip_info_lvl.drop("STATE", axis=1, inplace=True)
    zip_info_lvl.drop("agi_stub", axis=1, inplace=True)
    
    columns=zip_info_lvl.columns[1:]
    new_columns=["zipcode"]
    for column in columns:
        #print(index)
        new_columns.append(column+"-%d"%level)
    #print(columns[0]+"-%d"%level)
    #print(range(len(columns)))
    
    zip_info_lvl.columns=new_columns
    #print(zip_info_lvl.columns)
    
    train_data=train_data.merge(zip_info_lvl, how="left", on="zipcode")

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [58]:
train_data.shape

(23788, 1152)

In [59]:
columns=train_data.columns
for column in columns:
    train_data[column]=train_data[column].fillna(0)

In [63]:
train_data.drop("zipcode", axis=1, inplace=True)

In [64]:
train_data.to_csv("data/experiment_2.csv", index=False)